<a href="https://colab.research.google.com/github/jeanmhuang/etf-strategy-simulator/blob/main/ETF_risk_return_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance plotly

import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import datetime

# Define tickers and date range
etfs = {
    "S&P 500 (SPY)": "SPY",
    "Nasdaq 100 (QQQ)": "QQQ",
    "Gold (GLD)": "GLD",
    "Bonds (TLT)": "TLT",
    "Small Caps (IWM)": "IWM"
}
start_date = "2018-01-01"
end_date = datetime.date.today().strftime('%Y-%m-%d')

# Download price data with adjusted prices
data = yf.download(list(etfs.values()), start=start_date, end=end_date, auto_adjust=True)

# Handle multi-index format if present
if isinstance(data.columns, pd.MultiIndex):
    close = data['Close']
else:
    close = data[['Close']]  # fallback

# Daily returns
daily_returns = close.pct_change().dropna()

# Cumulative returns
cumulative = (1 + daily_returns).cumprod()

# Drawdowns
roll_max = close.cummax()
drawdowns = (close - roll_max) / roll_max

# Annualized return and volatility
annual_return = daily_returns.mean() * 252
annual_vol = daily_returns.std() * np.sqrt(252)

# --------- Plot 1: Cumulative Returns ---------
fig1 = go.Figure()
for col in cumulative.columns:
    fig1.add_trace(go.Scatter(x=cumulative.index, y=cumulative[col], name=col))
fig1.update_layout(title="Cumulative Returns (2018–Present)", xaxis_title="Date", yaxis_title="Growth of $1")
fig1.show()

# --------- Plot 2: Drawdowns ---------
fig2 = go.Figure()
for col in drawdowns.columns:
    fig2.add_trace(go.Scatter(x=drawdowns.index, y=drawdowns[col], name=col))
fig2.update_layout(title="Drawdowns", xaxis_title="Date", yaxis_title="Drawdown (%)")
fig2.show()

# --------- Plot 3: Risk vs Return ---------
fig3 = go.Figure(
    data=go.Scatter(
        x=annual_vol,
        y=annual_return,
        mode='markers+text',
        text=annual_return.index,
        textposition='top center'
    )
)
fig3.update_layout(title="Risk vs Return (Annualized)", xaxis_title="Volatility", yaxis_title="Annualized Return")
fig3.show()


[*********************100%***********************]  5 of 5 completed
